# Open AI Embedding 사용하기
ChatGPT로 유명한 OpenAI에서 제공하는 임베딩 기법입니다. 유료모델입니당

In [1]:
!pip install openai

   ---------------------------------------- 0.0/647.0 kB ? eta -:--:--
   ---------------------------------------- 647.0/647.0 kB 8.2 MB/s eta 0:00:00


In [29]:
from openai import OpenAI
import os

with open('data/my_api_key.txt', 'r') as file:
    API_KEY = file.read()

client = OpenAI(
    api_key=API_KEY
)
client

In [31]:
# OpenAI에서 제공하는 임베딩 모델 불러오기
embedding = client.embeddings.create(
    model = "text-embedding-ada-002",
    input = "야 저기 차 온다"
)
len(embedding.data[0].embedding)

1536

In [32]:
import pandas as pd

data = [
    "내일 차타고 놀러가자",
    "지금 오는 버스는 어디서 오는 버스야?",
    "차 한잔 하면서 이야기 하시죠",
    "5차 공동구매! 오늘만 세일!",
    "홍차 녹차 중에 어떤 차가 좋으세요?",
]

df = pd.DataFrame(data, columns=['text'])
df

,text
0,내일 차타고 놀러가자
1,지금 오는 버스는 어디서 오는 버스야?
2,차 한잔 하면서 이야기 하시죠
3,5차 공동구매! 오늘만 세일!
4,홍차 녹차 중에 어떤 차가 좋으세요?


In [33]:
def get_embedding(text):
    response = client.embeddings.create( input=text, model="text-embedding-ada-002")
    return response.data[0].embedding

df['embedding'] = df.apply(lambda row: get_embedding( row.text ), axis=1)

df

,text,embedding
0,내일 차타고 놀러가자,"[0.012128508649766445, -0.015187779441475868, ..."
1,지금 오는 버스는 어디서 오는 버스야?,"[0.008042754605412483, -0.011815354228019714, ..."
2,차 한잔 하면서 이야기 하시죠,"[0.016475515440106392, -0.023922652006149292, ..."
3,5차 공동구매! 오늘만 세일!,"[-0.01750493235886097, -0.0025386083871126175,..."
4,홍차 녹차 중에 어떤 차가 좋으세요?,"[0.004229011945426464, -0.011495853774249554, ..."


In [34]:
import numpy as np

def cos_sim(A, B):
  return A @ B/(np.linalg.norm(A)*np.linalg.norm(B))

def return_answer_candidate(df, query):
  '''
    df : 문장과 임베딩 벡터가 들어있는 데이터 프레임
    query : 질의할 문장
  '''
  query_embedding = get_embedding( query )
  df["similarity"] = df.embedding.apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))

  results_co = df.sort_values("similarity", ascending=False, ignore_index=True)
  return results_co.head(3)

In [35]:
return_answer_candidate(df, '야 저기 차온다')

,text,embedding,similarity
0,내일 차타고 놀러가자,"[0.012128508649766445, -0.015187779441475868, ...",0.878763
1,차 한잔 하면서 이야기 하시죠,"[0.016475515440106392, -0.023922652006149292, ...",0.872697
2,지금 오는 버스는 어디서 오는 버스야?,"[0.008042754605412483, -0.011815354228019714, ...",0.839820


In [36]:
return_answer_candidate(df, '예쁜 카페가고 싶어')

,text,embedding,similarity
0,내일 차타고 놀러가자,"[0.012128508649766445, -0.015187779441475868, ...",0.829990
1,차 한잔 하면서 이야기 하시죠,"[0.016475515440106392, -0.023922652006149292, ...",0.812928
2,홍차 녹차 중에 어떤 차가 좋으세요?,"[0.004229011945426464, -0.011495853774249554, ...",0.811625
